In [1]:
import os
import boto3
import sqlite3

In [34]:
def create_connection(db_file):
    """
    Creates a connection to the SQLite database
    :param db_file: Database file
    :return: Connection object or None 
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f'Connected to {db_file}')
    except Error as e:
        print(e)  

    return conn


def create_table(conn, overwrite=False):
    cursor = conn.cursor()
    # Create table if it does not exist
    cursor.execute(
        'CREATE TABLE IF NOT EXISTS viirs_alerts (' +
        'id INTEGER PRIMARY KEY AUTOINCREMENT, ' +
        'lat REAL, ' +
        'long REAL, ' +
        'frp REAL, ' +
        'acq_date TEXT, ' +
        'acq_time TEXT, ' +
        'confidence TEXT, ' +
        'daynight TEXT)'
    )
    cursor.close()
    # return cursor.lastrowid


def rows_from_file(file):
    import csv
    with open(file, 'r') as f:
        reader = csv.DictReader(f)
        to_db = [(row['latitude'], row['longitude'], row['frp'], row['acq_date'],
            row['acq_time'], row['confidence'], row['daynight']) for row in reader]
        # print(to_db)
    return to_db


# Insert rows (can also import rows from a file)
def insert_row(conn, file=None, values=None):
    sql = (
        f"INSERT INTO viirs_alerts (lat, long, frp, acq_date, acq_time, confidence, daynight) " +
        f"VALUES (?, ?, ?, ?, ?, ?, ?)"
    )
    cursor = conn.cursor()
    if file:
        db_values = rows_from_file(file)
        cursor.executemany(sql, db_values)
    else:
        cursor.execute(sql, values)
    # conn.commit()
    # print(cursor.execute('SELECT * FROM viirs_alerts').fetchall())
    cursor.close()
    # return cursor.lastrowid

# Test
# insert_row(conn, file='test.csv')

# Need to commit changes after running functions (i.e., conn.commmit())

In [3]:
# Create connection and cursor
conn = create_connection('viirs_alerts.db')
cursor = conn.cursor()

Connected to viirs_alerts.db


In [ ]:
# List of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print([table[0] for table in cursor.fetchall()])

In [ ]:
# List of columns in the database
cursor.execute("SELECT * FROM viirs_alerts")
[column[0] for column in cursor.description]

In [ ]:
# Add data to the table
import csv
with open('test.csv', 'r') as f:
    dr = csv.DictReader(f)
    to_db = [(r['latitude'], r['longitude'], r['frp'], r['acq_date'], r['acq_time'], r['confidence'], r['daynight']) for r in dr]
    print(to_db)

# cursor.executemany('INSERT INTO viirs_alerts (lat, long, frp, acq_date, acq_time, confidence, daynight) VALUES (?, ?, ?, ?, ?, ?, ?);', to_db)

# #
# conn.commit()

In [ ]:
# View data in the table
cursor.execute("SELECT * FROM viirs_alerts LIMIT 2")
print(cursor.fetchall())

In [ ]:
conn.close()

In [ ]:
s3_resource = boto3.resource('s3')

In [ ]:
bucket = s3_resource.Bucket('gfw-files')

In [ ]:
t = bucket.objects.filter(Prefix='jmaccarthy/Fires/VIIRS Archive')

In [ ]:
[file.key for file in t]

In [ ]:
# os.listdir('../VIIRS Archive')